<a href="https://colab.research.google.com/github/Ruheena-S/Hierarchical-classification-Loss-Functions-in-Image-Classification/blob/main/OvACascade_ResNet50_CIFAR100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install torch_optimizer torchmetrics
!nvidia-smi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 KB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 37.0 MB/s eta 0:00:00
Mon Mar 13 23:07:00 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    26W /  70W |      0MiB / 15360MiB |      0%    

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import sys
import torch.nn.functional as F
from torch_optimizer import Ranger
from torchvision import datasets
from torchvision import transforms,models
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
from torchmetrics import Accuracy
from torch.optim.lr_scheduler import CyclicLR

torch.manual_seed(42)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
#Tree Hierarchy of the labels

Tree = {"entity" : ["nature", "manmade"] , "nature":["flora", "bigAnimals", "smallAnimals", "aquaticAnimals", "scenes"], "manmade" : ["householdElectrical", "householdFurniture", "foodContainers", "vehicles1","vehicles2","largeManmadeOutdoorThings"],
        "flora": ["fruitsAndVegetables","flowers","trees"],"bigAnimals": ["largeCarnivores", "largeOmniAndHerbivores", "people", "mediumSizedMammals"],"smallAnimals":["smallMammals", "reptiles", "insects", "nonInsectInvertebrates"], "aquaticAnimals":["aquaticMammals", "fish"],"scenes":["cloud","forest","mountain","plain","sea"],
        "householdElectrical":["clock", "keyboard", "lamp", "telephone", "television"], "householdFurniture":["bed", "chair", "couch", "table", "wardrobe"],"foodContainers":["bottle", "bowl", "can", "cup", "plate"], "vehicles1":["bicycle", "bus", "motorcycle", "pickup_truck", "train"],"vehicles2":["lawn_mower", "rocket", "streetcar", "tank", "tractor"],"largeManmadeOutdoorThings":["bridge", "castle", "house", "road", "skyscraper"],
        "flowers":["orchid", "poppy", "rose", "sunflower", "tulip"],"fruitsAndVegetables":["apple", "mushroom", "orange", "pear", "sweet_pepper"],"trees":["maple_tree", "oak_tree", "palm_tree", "pine_tree", "willow_tree"],"largeCarnivores":["bear", "leopard", "lion", "tiger", "wolf"],"largeOmniAndHerbivores":["camel", "cattle", "chimpanzee", "elephant", "kangaroo"],"people":["baby", "boy", "girl", "man", "woman"],
        "mediumSizedMammals":["fox", "porcupine", "possum", "raccoon", "skunk"],"smallMammals":["hamster", "mouse", "rabbit", "shrew", "squirrel"],"reptiles":["crocodile", "dinosaur", "lizard", "snake", "turtle"],"insects":["bee", "beetle", "butterfly", "caterpillar", "cockroach"],"nonInsectInvertebrates":["crab", "lobster", "snail", "spider", "worm"],"aquaticMammals":["beaver", "dolphin", "otter", "seal", "whale"],
        "fish":["aquarium_fish", "flatfish", "ray", "shark", "trout"]}

labels = ["apple","aquarium_fish","baby","bear","beaver","bed","bee","beetle","bicycle","bottle","bowl","boy","bridge","bus","butterfly","camel","can","castle","caterpillar","cattle","chair","chimpanzee","clock","cloud","cockroach","couch","crab","crocodile","cup","dinosaur","dolphin","elephant","flatfish","forest","fox","girl","hamster","house","kangaroo","keyboard","lamp","lawn_mower","leopard","lion","lizard","lobster","man","maple_tree","motorcycle","mountain","mouse","mushroom","oak_tree","orange","orchid","otter","palm_tree","pear","pickup_truck","pine_tree","plain","plate","poppy","porcupine","possum","rabbit","raccoon","ray","road","rocket","rose","sea","seal","shark","shrew","skunk","skyscraper","snail","snake","spider","squirrel","streetcar","sunflower","sweet_pepper","table","tank","telephone","television","tiger","tractor","train","trout","tulip","turtle","wardrobe","whale","willow_tree","wolf","woman","worm", "flowers","fruitsAndVegetables", "trees", "largeCarnivores", "largeOmniAndHerbivores", "people", "mediumSizedMammals", "smallMammals", "reptiles", "insects", "nonInsectInvertebrates", "aquaticMammals", "fish", "flora", "bigAnimals", "smallAnimals","aquaticAnimals", "scenes", "householdElectrical", "householdFurniture", "foodContainers", "vehicles1", "vehicles2", "largeManmadeOutdoorThings", "nature" ,"manmade", "entity"]


In [ ]:
#Level-wise representation of labels

level = [None]*5
level[0] = ["entity"]
level[1] = ["nature", "manmade"]
level[2] = ["flora", "bigAnimals", "smallAnimals", "aquaticAnimals", "scenes", "householdElectrical", "householdFurniture","foodContainers", "vehicles1", "vehicles2","largeManmadeOutdoorThings"]
level[3] = ["flowers", "fruitsAndVegetables", "trees", "largeCarnivores", "largeOmniAndHerbivores", "people", "medimSizedMammals", "smallMammals", "reptiles", "insects", "nonInsectInvertebrates", "aquaticMammals", "fish", "cloud", "forest", "mountain", "plain", "sea", "clock", "keyboard", "lamp", "telephone", "television", "bed", "chair", "couch", "table", "wardrobe", "bottle", "bowl", "can", "cup", "plate", "bicycle", "bus", "motorcycle", "pickup_truck", "train", "lawn_mower", "rocket", "streetcar", "tank", "tractor", "bridge", "castle", "house", "road", "skyscraper"]
level[4] = ["orchid", "poppy", "rose", "sunflower", "tulip", "apple", "mushroom", "orange", "pear", "sweet_pepper", "maple_tree", "oak_tree", "palm_tree", "pine_tree", "willow_tree", "bear", "leopard", "lion", "tiger", "wolf", "camel", "cattle", "chimpanzee", "elephant", "kangaroo", "baby", "boy", "girl", "man", "woman", "fox", "porcupine", "possum", "raccoon", "skunk", "hamster", "mouse", "rabbit", "shrew", "squirrel", "crocodile", "dinosaur", "lizard", "snake", "turtle", "bee", "beetle", "butterfly", "caterpillar", "cockroach", "crab", "lobster", "snail", "spider", "worm", "beaver", "dolphin", "otter", "seal", "whale", "aquarium_fish", "flatfish", "ray", "shark", "trout"]



# Input    - class label of the CIFAR100 dataset
# Function - To find the level of the given class label
# output   - Returns an integer which represents the level of the class label and returns -1 if it doesn't belong to any level
def get_level(node):
  if (node in level[0]):
    return 0
  elif (node in level[1]):
    return 1
  elif (node in level[2]):
    return 2
  elif (node in level[3]):
    return 3
  elif (node in level[4]):
    return 4
  else:
    return -1

# Input    - class label of the CIFAR100 dataset
# Function - To find all the children of the given class label using the Tree hierarchy
# output   - Returns an list of the labels which are children and returns None if the given label is a leaf

def get_children(node):
  if node in Tree.keys():
    return Tree[node]    
  else:
    return None


# Input    - class label of the CIFAR100 dataset
# Function - To find the parent of the given class label using the level and children function.
# output   - Returns the parent of the class label
def get_parent(node):
  l = get_level(node)
  if l == 0: 
    return node
  for i in level[l-1]:
    if(get_children(i) is not None):
      if node in get_children(i):
        return i
  return None

# Input    - class label of the CIFAR100 dataset and the level of the label
# Function - finds ancestors of the the given label. 
# output   - Returns the label itself if given level is greater than the label's level else returns the ancestor at level l
def get_ancestor(node, l):
  h = get_level(node)
  if l >= h:
    return node
  y = node
  for i in range(h-l):
    y = get_parent(y)
  return y

# Input    - class label of the CIFAR100 dataset
# Function - To find the descendants of the given class label including the given class label
# output   - Returns an list of class labels which are descendants else empty list if the class label is a leaf.
def get_descendants(node):
  c = get_children(node)
  d = []
  if c is not None:
    for i in c:
      d.extend(get_descendants(i)) 
    return d
  else :
    return [node] 


# Input    - two class labels of the CIFAR100 dataset
# Function - Finds the distance between the given two nodes
# output   - Returns an integer which represents the distance between class labels
def tree_loss(node1,node2):
  l1 = get_level(node1)
  l2 = get_level(node2)
  l = min(l1,l2)
  while l >= 0:
    if get_ancestor(node1,l) == get_ancestor(node2,l):
      break
    else :
      l = l-1
  return ((l1 + l2) - (2*l))
  

In [ ]:

def target_trans(target):
  y = -torch.ones(127)
  y[target] = 1

  global labels
  label = labels[target]
  for j in range(27):
    if label in get_descendants(labels[100+j]):
      y[100+j] = 1

  return y,target

In [ ]:
# Define transforms for the dataset
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

dataset = datasets.CIFAR100(root='data/', train=True, download=True, transform=transform_train,target_transform=target_trans)
test_dataset = datasets.CIFAR100(root='data/', train=False, download=True, transform=transform_test,target_transform=target_trans)


val_size = 5000
train_size = len(dataset) - val_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])
batch_size=256

train_loader = DataLoader(train_ds, batch_size, shuffle=True, num_workers=4)
valid_loader = DataLoader(val_ds, batch_size, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size, num_workers=4)

  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting data/cifar-100-python.tar.gz to data/
Files already downloaded and verified


/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
# Input    - output : Predicted output and target : True output of the input
# Function - For each batch of inputs, finds the tree loss using OvA algorithm.
# output   - Returns the floating integer by calculating the batch-wise tree loss

def tLoss(output,target):
  
  batch_size = target.size(0)
  num_classes = output.size(1)

  # Height of the hierarchy tree
  h = 4
  tau_values=[0,0,0,0,0]
  val = None
  pred = None
  loss = 0
  global labels

  # Iterate each example in the batch
  for i in range(batch_size):
    t = target[i]

    h = 4
    # searching int the bottom-up manner in the hierarchy tree
    while h >= 0:

      # Clone the output array to check at every level
      values = output.clone().detach()

      # Iterate over each class
      for j in range(num_classes):

        level = get_level(labels[j])

        # Values of nodes below the current level are considered in the ancestor hence we assign the min value
        if level > h:
          values[i,j] = -10000

        elif level < h:
          # Similary for the node which is above current level but is not a leaf node, we assign the min value
          if get_children(labels[j]) is not None:
            values[i,j] = -10000   

      val,pred = torch.max(values[i,:],0)

      # If the output is greater than the tau value at each level we consider the pred value assigned in the previous step
      if h == 4 and val >= tau_values[4]:
        break
      if h == 3 and val >= tau_values[3]:
        break
      if h == 2 and val >= tau_values[2]:
        break
      if h == 1 and val >= tau_values[1]:
        break
      if h == 0 and val >= tau_values[0]:
        break 
        
      h = h - 1
    
    loss = loss + tree_loss(labels[t],labels[pred])

  loss = loss/batch_size
  # Average loss for the batch is returned
  return loss


In [ ]:
class Hinge_Loss(torch.nn.Module):
    
    def __init__(self):
        super(Hinge_Loss,self).__init__()
        
    def forward(self,x,y):

        temp = (1 - (x * y))
        clamp = F.relu(temp)
        total_loss = torch.sum(clamp)/x.size(0)
        return total_loss

In [ ]:
num_classes = 127
num_epochs = 50

In [ ]:
# Define the ResNet50 model and freeze layers

model = models.resnet50(pretrained=True)
model.conv1 = nn.Conv2d(3, 64, kernel_size = (3,3), padding = (1, 1), bias = False)
model.maxpool = nn.Identity()

for param in model.parameters():
    param.requires_grad = True
# for param in model.layer4.parameters():
#     param.requires_grad = True

model.fc = nn.Linear(2048, 127)

model.to(device)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): Identity()
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentu

In [ ]:
# Define loss function and optimizer

criterion = Hinge_Loss().to(device)
accuracy = Accuracy(task="multiclass",num_classes=num_classes).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
# optimizer = Ranger(model.parameters(), lr=1e-3, weight_decay=1e-4) 
scheduler = CyclicLR(optimizer, base_lr=1e-6, max_lr=1e-3, step_size_up=len(train_loader)//2, cycle_momentum=False)

In [ ]:
def get_accuracy(output, target):
  batch_size = target.size(0)

  pred = output[:,0:100]      # considering the first 100 of the 127 nodes because they are the true classes(No internal node)
  pred = pred.max(dim=1)[1]

  correct = pred==target
  acc = correct.float().sum(0)

  return acc/batch_size

In [ ]:
#Training

total_train_step = len(train_loader)
#print(total_train_step)
total_val_step=len(valid_loader)
BEST_VAL_METRIC = 0
BEST_MODEL = None


for epoch in range(1, num_epochs+1):

    train_loss=0
    train_acc=0.0
    model.train()

    for i, (images, target) in enumerate(train_loader, 1):

        y_trans = target[0]
        y_true = target[1]

        # Move tensors to the configured device
        images = images.to(device)
        y_true = y_true.to(device)
        y_trans = y_trans.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, y_trans)

        train_loss += loss
        train_acc += get_accuracy(outputs, y_true)
        #train_acc += accuracy(outputs[:,0:100], y_true)
        
        
        # Backward and optimize
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        # scheduler.step(train_loss/total_train_step)
        

    print(f'Epoch [{epoch}/{num_epochs}] - Loss: {(train_loss/total_train_step):.4f}, Accuracy: {(train_acc/total_train_step):.4f}')

    model.eval() 
    # Validation
    with torch.no_grad():
        val_acc = 0
        val_loss=0
        Tree_Loss_Value = 0
        for i, (images, target) in enumerate(valid_loader, 1):

            y_trans = target[0]
            y_true = target[1]

            # Move tensors to the configured device
            images = images.to(device)
            y_true = y_true.to(device)
            y_trans = y_trans.to(device)

            outputs = model(images)
            val_loss += criterion(outputs, y_trans)
            Tree_Loss_Value += tLoss(outputs,y_true)
            val_acc += get_accuracy(outputs, y_true)
            #val_acc += accuracy(outputs[:,0:100], y_true)

    if val_acc/total_val_step > BEST_VAL_METRIC:
        BEST_VAL_METRIC = val_acc/total_val_step
        BEST_MODEL = model.state_dict() 
        torch.save(model, "/content/drive/MyDrive/MTP_Phase2/saved_models/resnet50_OVACascade_cifar100_RangerOPT.pth")

    print(f'Accuracy of the network on validation images: {(val_acc/total_val_step):.4f}, loss: {(val_loss/total_val_step):.4f}, Tree loss: {(Tree_Loss_Value/total_val_step):.4f}') 

Epoch [1/50] - Loss: 5.9850, Accuracy: 0.1079
Accuracy of the network on validation images: 0.1824, loss: 5.5845, Tree loss: 2.8938
Epoch [2/50] - Loss: 5.4309, Accuracy: 0.1979
Accuracy of the network on validation images: 0.3058, loss: 5.0627, Tree loss: 2.6974
Epoch [3/50] - Loss: 5.0559, Accuracy: 0.2618
Accuracy of the network on validation images: 0.3470, loss: 4.7409, Tree loss: 2.5333
Epoch [4/50] - Loss: 4.8064, Accuracy: 0.3034
Accuracy of the network on validation images: 0.3845, loss: 4.5585, Tree loss: 2.4496
Epoch [5/50] - Loss: 4.5196, Accuracy: 0.3511
Accuracy of the network on validation images: 0.4261, loss: 4.2333, Tree loss: 2.2872
Epoch [6/50] - Loss: 4.2404, Accuracy: 0.3891
Accuracy of the network on validation images: 0.4568, loss: 4.0121, Tree loss: 2.1715
Epoch [7/50] - Loss: 4.0291, Accuracy: 0.4300
Accuracy of the network on validation images: 0.4872, loss: 3.8153, Tree loss: 2.0823
Epoch [8/50] - Loss: 3.8306, Accuracy: 0.4641
Accuracy of the network on val

In [ ]:

#Testing
model.load_state_dict(BEST_MODEL)

total_test_step=len(test_loader)

with torch.no_grad():
    test_acc=0
    test_loss=0
    Tree_Loss_Value=0

    for i, (images, target) in enumerate(test_loader, 1):
        
        y_trans = target[0]
        y_true = target[1]
        
        images = images.to(device)
        y_true = y_true.to(device)
        y_trans = y_trans.to(device)

        # Forward pass
        outputs = model(images)
        
        # Loss
        test_loss += criterion(outputs,y_trans)
        Tree_Loss_Value += tLoss(outputs,y_true)
        test_acc += get_accuracy(outputs, y_true)
        #test_acc += accuracy(outputs[:,0:100], y_true)

    print(f'Accuracy of the network on test images: {(test_acc/total_test_step):.4f}, loss: {(test_loss/total_test_step):.4f}, Tree loss: {(Tree_Loss_Value/total_test_step):.4f}')

Accuracy of the network on test images: 0.7405, loss: 2.2062, Tree loss: 0.9994
